In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
from art_skills import SlnStrokeExpression

$$ \Lambda(t) = \frac{1}{\sigma\sqrt{2\pi}(t-t_0)} \exp{\left(-\frac{(\ln(t-t_0)-\mu)^2}{2\sigma^2}\right)} $$

In [ ]:
def evaluate_speed(t, t0, D, th1, th2, sigma, mu):
    if False:   # use c++ wrapped version
        sln_stroke = SlnStrokeExpression(t0, D, th1, th2, sigma, mu)
        return np.array([sln_stroke.speed(t_) for t_ in t])
    else:       # use numpy version
        return D / (sigma * np.sqrt(2 * np.pi) * (t - t0)) * np.exp(-np.square(np.log(t - t0) - mu) /
                                                                 (2 * sigma * sigma))

In [ ]:
def plot_sln_single(t0, D, th1, th2, sigma, mu):
    t = np.logspace(-5, 1, 100)
    v = evaluate_speed(t, t0, D, th1, th2, sigma, mu) * (t-t0)
    plt.figure(figsize=(12, 4))
    plt.subplot(131)
    plt.plot(np.log(t - t0), v)
    plt.xlim(-2.5, 2.5)
    plt.ylim(0, 1)
    plt.title('Gaussian Curve: Velocity vs ln(t-t0)')
    plt.subplot(132)
    plt.plot(t, v)
    plt.ylim(0, 1)
    plt.xlim(0, 0.6)
    plt.title('Velocity vs Time')
    plt.subplot(133)
    plt.plot(t, np.cumsum(v) - v[0])
    plt.xlim(0, 0.6)
    plt.ylim(0, 100)
    plt.title('Position vs Time')
    plt.show()

In [ ]:
interactive_plot = widgets.interactive(plot_sln_single,
                                       t0=widgets.FloatSlider(min=-5, max=-1e-12, value=-0.1),
                                       D=widgets.FloatSlider(min=0, max=5, value=1),
                                       th1=widgets.fixed(0),
                                       th2=widgets.fixed(0),
                                       sigma=widgets.FloatSlider(min=1e-12, max=2, value=1),
                                       mu=widgets.FloatSlider(min=-5, max=5, value=0))

output = interactive_plot.children[-1]
output.layout.height = '350px'
interactive_plot


In [ ]:
def plot_sln_array(ax, t0, D, th1, th2, sigma, mu):
    t = np.logspace(-5, 1, 100)
    v = evaluate_speed(t, t0, D, th1, th2, sigma, mu)
    ax.plot(t, v)
    ax.set_ylim(0, 1)
    ax.set_xlim(0, 10)

In [ ]:
sigma_vals = np.linspace(1e-10, 2, 5)
mu_vals = np.linspace(0, 3, 6)
fig, axes = plt.subplots(len(sigma_vals), len(mu_vals), sharex=True, sharey=True, figsize=(20, 14))
for ri, sigma in enumerate(sigma_vals):
    for ci, mu in enumerate(mu_vals):
        plot_sln_array(axes[ri][ci], 0, 1, 0, 0, sigma, mu)
        axes[0][ci].set_title('$\\mu = {:}$'.format(mu))
    axes[ri][0].set_ylabel('$\\sigma = {:.1f}$'.format(sigma))

In [ ]:
# Generate ground truth for test_sln_stroke_fit
dt = 0.01
t = np.arange(0, 0.6 + dt, dt)
params = np.array([-0.8, 1, 0., 0., 0.2, 0.1])
v = evaluate_speed(t, *params)
x = np.cumsum(v * dt)
x = x - x[0]
# Print out unit test "expected"
t_expected = np.array([0, 0.2, 0.4, 0.6]).reshape(-1, 1)
x_expected = np.array([x[t == t_expected_][0] for t_expected_ in t_expected]).reshape(-1, 1)
data_expected = np.hstack((t_expected, x_expected, np.zeros(t_expected.shape)))
print('pExpected = np.array(', params.tolist(), ')')
print('data = np.array(', data_expected.tolist(), ')')

In [ ]:
# Visualize Unit Test Results
def plot_sln_test(t0, D, th1, th2, sigma, mu, ls='k-'):
    t = np.logspace(-3, 10**0.6, 400)
    v = evaluate_speed(t, t0, D, th1, th2, sigma, mu)
    plt.subplot(131)
    plt.plot(np.log(t - t0), v, ls)
    plt.xlim(-2.5, 2.5)
    plt.title('Gaussian Curve: Velocity vs ln(t-t0)')
    plt.subplot(132)
    plt.plot(t, v, ls)
    plt.xlim(0, 0.6)
    plt.title('Velocity vs Time')
    plt.subplot(133)
    plt.plot(t, np.cumsum(v) - v[0], ls)
    plt.xlim(0, 0.6)
    plt.title('Position vs Time')
plt.figure(figsize=(12, 4))
plot_sln_test(*params, 'k-')
plot_sln_test(-0.91072802, 0.99922022, 0., 0., 0.18157826, 0.19371412, 'r:')
plt.show()